# Homework 13

In [151]:
import os
import re
import pandas as pd
import numpy as np

In [152]:
cwd = os.getcwd()
fn = os.path.join(cwd, 'dates.txt')
with open(fn, 'r') as f:
    content = f.read()
print(content)

03/25/93 Total time of visit (in minutes):
6/18/85 Primary Care Doctor:
sshe plans to move as of 7/8/71 In-Home Services: None
7 on 9/27/75 Audit C Score Current:
2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7
.Per 7/06/79 Movement D/O note:
4, 5/18/78 Patient's thoughts about current substance abuse:
10/24/89 CPT Code: 90801 - Psychiatric Diagnosis Interview
3/7/86 SOS-10 Total Score:
(4/10/71)Score-1Audit C Score Current:
(5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC_12.6Activities of Daily Living (ADL) Bathing: Independent
4/09/75 SOS-10 Total Score:
8/01/98 Communication with referring physician?: Done
1/26/72 Communication with referring physician?: Not Done
5/24/1990 CPT Code: 90792: With medical services
1/25/2011 CPT Code: 90792: With medical services
4/12/82 Total time of visit (in minutes):
1; 10/13/1976 Audit C Score, Highest/Date:
4, 4/24/98 Relevant Drug History:
) 59 yo unemployed w referred by Urgent Care for psychiatric evaluation and follow up. The pat

The Task is to extract relevant infromation from the messy medical data using regex.

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

1) Extract the date strings
Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

2) Normalize the extracted dates considering the following rules:  
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).

3) Sort records in ascending chronological order accoring.




In [153]:
def init_df():
    spl_content = content.split('\n')
    return pd.DataFrame(spl_content, columns=['text']) 
df =  init_df()
df = df.drop(500, axis=0)
pd.set_option('display.max_rows', 500, 'max_colwidth', 200)
df.loc[487]

text    appendectomy s/p delivery 1992 Prior relevant labs:
Name: 487, dtype: object

In [208]:
df_ext = df['text'].str.extract(r'(?P<Month>\d{1,2})[/|-](?P<Day>\d{1,2})[/|-](?P<Year>\d{2,4})')
df_ext = df_ext.combine_first(df['text'].str.extract(r'(?P<Day>\d{1,2} )(?P<Month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?P<Year> \d{2,4})'))
df_ext = df_ext.combine_first(df['text'].str.extract(r'(?P<Month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,|.]?(?P<Day> \d{2})[.|,]? (?P<Year>\d{2,4})'))
df_ext = df_ext.combine_first(df['text'].str.extract(r'(?P<Month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,|.]? (?P<Year>\d{2,4})'))
df_ext = df_ext.combine_first(df['text'].str.extract(r'(?P<Month>\d{1,2})/(?P<Year>\d{2,4})'))
df_ext = df_ext.combine_first(df['text'].str.extract(r'(?P<Year>\d{4})'))
df_ext

,Day,Month,Year
0,25,03,93
1,18,6,85
2,8,7,71
3,27,9,75
4,6,2,96
5,06,7,79
6,18,5,78
7,24,10,89
8,7,3,86
9,10,4,71


In [220]:
df_ext = df_ext.replace(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                       ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'])
for i, row in df_ext.iterrows():
    if pd.isnull(df_ext.loc[i, 'Day']):
        df_ext.at[i, 'Day'] = '01'
    elif len(df_ext.loc[i, 'Day']) == 1:
        df_ext.at[i, 'Day'] = '0' + df_ext.loc[i, 'Day']
    if len(str(df_ext.loc[i, 'Month'])) == 1:
        df_ext.at[i, 'Month'] = '0' + str(df_ext.loc[i, 'Month'])
    elif pd.isnull(df_ext.loc[i, 'Month']):
        df_ext.at[i, 'Month'] = '01'
    if len(str(df_ext.loc[i, 'Year'])) == 2:
        df_ext.at[i, 'Year'] = '19' + str(df_ext.loc[i, 'Year'])
data = pd.DataFrame(pd.to_datetime(df_ext), columns=['res'])
data['index'] = data.index
data = data.sort_values(by='res')
data.reset_index(drop=True) 

,res,index
0,1971-04-10,9
1,1971-05-18,84
2,1971-07-08,2
3,1971-07-11,53
4,1971-09-12,28
5,1972-01-01,474
6,1972-01-13,153
7,1972-01-26,13
8,1972-05-06,129
9,1972-05-13,98
